# Load imports and data

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import missingno as msno


In [ ]:
    # Do analysis in all data:
    # (Safe because the rest of the notebook takes into account dates and whether the data is dev or comp, allowing it to keep the data separated)
    # (It has been confirmed that this option produces the same results as running the other two options separately)
DO_ANALYSIS = "ALL-DATA"

    # Only do analysis on development data:
    # (In order to reproduce data exploration from the report, run this script with only the development data)
# DO_ANALYSIS = "DEV-DATA"

    # Only run analysis on competition data:
# DO_ANALYSIS = "COMP-DATA"

In [ ]:
accounts = pd.read_csv('data/account.csv', sep=';')
clients = pd.read_csv('data/client.csv', sep=';')
disp = pd.read_csv('data/disp.csv', sep=';')
districts = pd.read_csv('data/district.csv', sep=';')

if DO_ANALYSIS == "ALL-DATA":
    print("Running analysis on all data")
    cards = pd.read_csv('data/card.csv', sep=';')
    loans = pd.read_csv('data/loan.csv', sep=';')
    trans = pd.read_csv('data/trans.csv', sep=';')
elif DO_ANALYSIS == "DEV-DATA":
    print("Running analysis on development data")
    cards = pd.read_csv('data/card_dev.csv', sep=';')
    loans = pd.read_csv('data/loan_dev.csv', sep=';')
    trans = pd.read_csv('data/trans_dev.csv', sep=';')
elif DO_ANALYSIS == "COMP-DATA":
    print("Running analysis on competition data")
    cards = pd.read_csv('data/card_comp.csv', sep=';')
    loans = pd.read_csv('data/loan_comp.csv', sep=';')
    trans = pd.read_csv('data/trans_comp.csv', sep=';')

# Data Understanding

In [ ]:
loans

In [ ]:
def info(table, name):
    print("==============")
    print(name)
    print("--------------")
    print(table.info(verbose=True, max_cols=1000, memory_usage=False, show_counts=True))
    print("--------------")
    print("Description of the table (table.describe()). For some columns, such as ids, these metrics are not useful and should be ignored.")
    print(table.describe())
    print("--------------")
    for column in table.columns:
        values = table[column].unique()
        if len(values) < 10:
            print(f"\tColumn {column} can have values {values}")
        if "?" in values:
            print(f"\tColumn {column} has at least one value '?'")
    print("\n\n")
    
info(accounts, "accounts")
info(cards, "cards")
info(clients, "clients")
info(disp, "disp")
info(districts, "districts")
info(loans, "loans")
info(trans, "transactions")


In [ ]:
districts[districts['unemploymant rate \'95 '] == '?']

In [ ]:
districts[districts['no. of commited crimes \'95 '] == '?']

- Null values: Only transactions_dev and district have null values
    - transactions_dev: operation, k_symbol, bank, account
    - district: unemploymant rate '95, no. of commited crimes '95

- Dates: We should confirm that for a loan in a given date we only consider accounts/clients/transactions/etc from before that date, using the respective date fields

### Collected Data

- account.csv - relation account (4500 objects) 
- client.csv - relation client (5369 objects)
- disp.csv - relation disposition (5369 objects) 
- district.csv - relation demographic data (77 objects)
- ~~relation permanent order (0/6471 objects)~~
- trans_dev.csv - relation transaction (396685/1056320 objects)
- card_dev.csv - relation credit card (177/892 objects)
- loan_dev.csv - relation loan (328/682 objects)


accounts : 4500 objects
 #  | Column    |   Non-Null Count | type 
--- | ------     |  -------------- | ----- 
 0 |  account_id  | 4500 non-null |  PRIMARY KEY 
 1  | district_id|  4500 non-null |  FOREIGN KEY 
 2  | frequency  |  4500 non-null |  enum('monthly issuance', 'issuance after transaction', 'weekly issuance')
 3  | date       |  4500 non-null |  YYMMDD


clients : 5369 objects
 #  | Column        |Non-Null Count|  type
--- | ------        |--------------|  -----
 0  | client_id     |5369 non-null |  PRIMARY KEY
 1  | birth_number | 5369 non-null |  YYMMDD + 5000 if woman
 2   |district_id |  5369 non-null |  FOREIGN KEY


disp : 5369 objects
 #  | Column     | Non-Null Count | type 
--- | ------     | -------------- | ----- 
 0  | disp_id    | 5369 non-null  | PRIMARY KEY 
 1  | client_id  | 5369 non-null  | FOREIGN KEY 
 2  | account_id | 5369 non-null  | FOREIGN KEY 
 3  | type       | 5369 non-null  | enum('OWNER', 'DISPONENT')


districts : 77 objects
 #  | Column                                            | Non-Null Count | type  
--- | ------                                            | -------------- | -----  
 0  | code                                              | 77 non-null    | PRIMARY KEY  
 1  | name                                              | 77 non-null    | UNIQUE KEY 
 2  | region                                            | 77 non-null    | enum('Prague', 'central Bohemia', 'south Bohemia', 'west Bohemia', 'north Bohemia', 'east Bohemia', 'south Moravia', 'north Moravia')
 3  | no. of inhabitants                                | 77 non-null    | int  
 4  | no. of municipalities with inhabitants < 499      | 77 non-null    | int  
 5  | no. of municipalities with inhabitants 500-1999   | 77 non-null    | int  
 6  | no. of municipalities with inhabitants 2000-9999  | 77 non-null    | int  
 7  | no. of municipalities with inhabitants >10000     | 77 non-null    | int  
 8  | no. of cities                                     | 77 non-null    | int  
 9  | ratio of urban inhabitants                       |  77 non-null    | float
 10 | average salary                                  |   77 non-null    | int (currency unit)  
 11 | unemploymant rate '95                          |    77 non-null    | float 
 12 | unemploymant rate '96                         |     77 non-null    | float
 13 | no. of enterpreneurs per 1000 inhabitants    |      77 non-null    | int  
 14 | no. of commited crimes '95                  |       77 non-null    | int 
 15 | no. of commited crimes '96                 |        77 non-null    | int  

transactions : 396685/1056320 objects
 #  | Column    |  Non-Null Count |  type  
--- | ------    |  -------------- |  -----  
 0  | trans_id  |  396685 non-null|  PRIMARY KEY  
 1  | account_id|  396685 non-null|  FOREIGN KEY  
 2  | date      |  396685 non-null|  YYMMDD
 3  | type      |  396685 non-null|  enum('credit', 'withdrawal', 'withdrawal in cash') 
 4  | operation |  325924 non-null|  enum('credit in cash', 'collection from another bank', 'withdrawal in cash', 'remittance to another bank', 'credit card withdrawal')
 5  | amount    |  396685 non-null|  float64 (currency unit)
 6  | balance   |  396685 non-null|  float64 (currency unit)
 7  | k_symbol  |  211441 non-null|  enum('interest credited', ' ', 'household', 'payment for statement', 'insurrance payment', 'sanction interest if negative balance', 'old-age pension')
 8  | bank      |  97242 non-null |  string (other bank ID)
 9  | account   |  102229 non-null|  account (other bank's account ID)

- type="withdrawal in cash" is redundant with operation="withdrawal in cash"

cards : 177/892 objects
 #  | Column  | Non-Null Count | type 
--- | ------  | -------------- | ----- 
 0  | card_id | 177 non-null   | PRIMARY KEY 
 1  | disp_id | 177 non-null   | FOREIGN KEY 
 2  | type    | 177 non-null   | enum('classic', 'junior', 'gold')
 3  | issued  | 177 non-null   | YYMMDD 

loans_dev : 328/682 objects
 #  | Column     | Non-Null Count | type
--- | ------     | -------------- | -----
 0  | loan_id    | 328 non-null   | PRIMARY KEY
 1  | account_id | 328 non-null   | FOREIGN KEY
 2  | date       | 328 non-null   | YYMMDD
 3  | amount     | 328 non-null   | derived(duration*payments)
 4  | duration   | 328 non-null   | int (number of months)
 5  | payments   | 328 non-null   | int (currency unit/month)
 6  | **status** |     328 non-null  |  TARGET (1 = paid, -1 = not paid)

- binarize status (1 = not paid, 0 = paid)
- encode duration by year (is by month at the moment, but always multiples of 12)


### Check table association multiplicities

In [ ]:
# Account -*----------1-> District

# How many accounts has each district

accounts_count_per_district_array = accounts.value_counts('district_id').sort_index().array
accounts_count_per_district = { name: accounts_count_per_district_array[idx - 1] for idx, name in districts[['code ', 'name ']].values}


accounts_count_per_region = {} 
for idx, region in districts[['code ', 'region']].values:
    accounts_count_per_region[region] = accounts_count_per_region.get(region, 0) + accounts_count_per_district_array[idx - 1]

def dict_barplot(dict):
    """From a dict {x:y} it plots a barplot"""
    lenx = len(dict.keys())
    plt.figure(figsize=(lenx/4, 4))
    plt.bar(range(lenx), list(dict.values()), align='center')
    plt.xticks(range(lenx), list(dict.keys()), rotation=90)
    plt.show()

dict_barplot(accounts_count_per_district)
dict_barplot(accounts_count_per_region)

In [ ]:
# Client -*----------1-> District

# How many clients has each district

client_count_per_district_array = clients.value_counts('district_id').sort_index().array
client_count_per_district = { name: client_count_per_district_array[idx - 1] for idx, name in districts[['code ', 'name ']].values}


client_count_per_region = {} 
for idx, region in districts[['code ', 'region']].values:
    client_count_per_region[region] = client_count_per_region.get(region, 0) + client_count_per_district_array[idx - 1]

dict_barplot(client_count_per_district)
dict_barplot(client_count_per_region)

In [ ]:
# Account <-1----------1..2- Disposition -1----------1-> Client


clients_dict = {}
for idx, client, account, type in disp[['disp_id', 'client_id', 'account_id', 'type']].values:
    if client in clients_dict:
        print("DUPLICATE CLIENT", clients_dict[client])
        # THERE ARE NO DUPLICATE CLIENTS: client_id is just as unique as disp_id
        
    clients_dict[client] = clients_dict.get(client, 0) + 1

print("No clients missing from disposition:", set(clients['client_id'].values) == set(disp['client_id'].values))
print("No accounts missing from disposition:", set(accounts['account_id'].values) == set(disp['account_id'].values))

# EVERY ACCOUNT HAS 1 OWNER AND 0..1 DISPONENTS
for account in set(accounts['account_id'].values):
    disp_row = disp[disp['account_id'] == account]
    types = disp_row['type'].value_counts()
    if types.get('OWNER', 0) != 1:
        print("Account", account, "has", types.get('OWNER', 0), "owners")
    if types.get('DISPONENT', 0) != 0 and types.get('DISPONENT', 0) != 1:
        print("Account", account, "has", types.get('DISPONENT', 0), "disponents")

In [ ]:
# acc_district != owner_district != disponent_district
for account, acc_district in accounts[['account_id', 'district_id']].values:
    disp_row = disp[disp['account_id'] == account]
    owner_id = disp_row[disp_row['type'] == 'OWNER']['client_id'].values[0]
    owner = clients[clients['client_id'] == owner_id]
    owner_district = owner['district_id'].values[0]

    disponent_id = disp_row[disp_row['type'] == 'DISPONENT']
    if disponent_id.shape[0] == 1:
        disponent_id = disponent_id['client_id'].values[0]
        disponent = clients[clients['client_id'] == disponent_id]
        disponent_district = disponent['district_id'].values[0]
        print(acc_district, owner_district, disponent_district)
    else:
        print(acc_district, owner_district)


In [ ]:
# Card -0..1----------1-> Disponent
disponents_dict = {}
for idx, disponent in cards[['card_id', 'disp_id']].values:
    if disponent in disponents_dict:
        print("DUPLICATE DISPONENT", disponents_dict[disponent])
        # THERE ARE NO DUPLICATE DISPONENTS: disp_id is just as unique as card_id
    disponents_dict[disponent] = disponents_dict.get(disponent, 0) + 1

# There are disp without cards
print("No disps missing from cards:", set(disp['disp_id'].values) == set(cards['disp_id'].values))

In [ ]:
# Transaction -0..*----------1-> Account
print("No accounts missing from transactions:", set(accounts['account_id'].values) == set(trans['account_id'].values))

accounts_dict = {account: 0 for account in accounts['account_id'].values}
for idx, account in trans[['trans_id', 'account_id']].values:
    accounts_dict[account] = accounts_dict.get(account, 0) + 1

print("Accounts in dev have between", min(accounts_dict.values()), "and", max(accounts_dict.values()), "transactions")


In [ ]:
# Loan -0..1----------1-> Account
accounts_dict = {}
for idx, account in loans[['loan_id', 'account_id']].values:
    if account in accounts_dict:
        print("DUPLICATE ACCOUNT", accounts_dict[account])
        # THERE ARE NO DUPLICATE ACCOUNTS: account_id is just as unique as loan_id
    accounts_dict[account] = accounts_dict.get(account, 0) + 1

# There are accounts without loans
print("No accounts missing from loans:", set(accounts['account_id'].values) == set(loans['account_id'].values))

### Plots and statistics

In [ ]:
accounts['frequency'].hist()

In [ ]:
disp['type'].hist()

In [ ]:
# x labels rotated 90º
g = sns.histplot(districts, x='region')
_ = g.set_xticklabels(g.get_xticklabels(), rotation=90)


In [ ]:
# Correlation of missing values (how much a value missing in one column is correlated with a value being missing in another column)
msno.heatmap(trans)

# Bank and account very correlated, as expected (both are missing at the same time)

In [ ]:
cards['issued'].hist()

In [ ]:
sns.scatterplot(trans, x='amount', y='balance', hue='type')

# Data Preparation

Also includes the parts of data understanding and exploration which require some preparation.

### District

In [ ]:
# Consider '?' as NaN and treat respetive columns as floats, so we can plot the correlation matrix
districts.replace('?', np.nan, inplace=True)
districts['no. of commited crimes \'95 '] = districts['no. of commited crimes \'95 '].astype(float)
districts['unemploymant rate \'95 '] = districts['unemploymant rate \'95 '].astype(float)

In [ ]:
def corr_plot(df, size=(13, 9), annot=True):
    # Compute the correlation matrix
    corr = df.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=size)

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=annot, fmt=".2f")

corr_plot(districts.drop(columns=['code ', 'region', 'name ']))

# High correlation between number of inhabitants and number of commited crimes
# Also between number of commited crimes in different years
# Also between number of unemployment rate in different years
# Also between average salary and commited crimes

In [ ]:
# Correlation between number of commited crimes and number of inhabitants suggests that 
# crimes per capita would give more useful information
districts['no. of commited crimes \'95 '] = districts['no. of commited crimes \'95 '] / districts['no. of inhabitants']
districts['no. of commited crimes \'96 '] = districts['no. of commited crimes \'96 '] / districts['no. of inhabitants']

In [ ]:
sns.histplot(np.log(districts['no. of inhabitants']), kde=True)

In [ ]:
# Since we only have one row with missing values, it seams reasonable to complete the missing 
# values of unemployment rate and commited crimes with the mean values of the other rows
districts['unemploymant rate \'95 '].fillna(districts['unemploymant rate \'95 '].mean(), inplace=True)
districts['no. of commited crimes \'95 '].fillna(districts['no. of commited crimes \'95 '].mean(), inplace=True)

# Note: the districts table is the same for dev and comp (only one value), so it is safe to do the mean, as there is 
# no extra comp data that was missing in the dev data

In [ ]:
districts.drop(['code ', 'region', 'name '], axis=1).describe()

In [ ]:
districts

### Merge tables

In [ ]:
# Merge the tables based on the foreign keys, in order to have one row per loan with all relevant information for that loan

# Merge loans and accounts and with the account district information
df = pd.merge(loans, accounts, on='account_id', how='left', suffixes=('_loan', '_account'), validate='one_to_one')
df = pd.merge(df, districts, left_on='district_id', right_on='code ', how='left', suffixes=(None, '_account_district'), validate='many_to_one')

# Merge dispositions with clients and card information
reduced_disp = pd.merge(disp, clients, on='client_id', how='left', suffixes=('_disp', '_client'), validate='one_to_one')
reduced_disp = pd.merge(reduced_disp, cards, on='disp_id', how='left', suffixes=('_disp', '_card'), validate='one_to_one')
# Missing cards replaced with "no card"
reduced_disp['type_card'] = reduced_disp['type_card'].fillna('no card')
# Merge that with the client district information
reduced_disp = pd.merge(reduced_disp, districts, left_on='district_id', right_on='code ', how='left', suffixes=(None, '_client_district'), validate='many_to_one')

# Filter by owners
owners = reduced_disp[reduced_disp['type_disp'] == 'OWNER']
owners.columns = owners.columns.map(lambda x: str(x) + '_owner' if x != 'account_id' else x)

# Filter by the other disponent
disponents = reduced_disp[reduced_disp['type_disp'] == 'DISPONENT']
disponents.columns = disponents.columns.map(lambda x: str(x) + '_disponent' if x != 'account_id' else x)

# Merge the owners and other disponents with the loans
df = pd.merge(df, owners, on='account_id', how='left', suffixes=(None, '_something_wrong'), validate='one_to_one')
df = pd.merge(df, disponents, on='account_id', how='left', suffixes=(None, '_something_wrong'), validate='one_to_one')

### Create new features based on transactions data

In [ ]:
# This takes into account

def filter_relevants(trans_dev, account_id, date):
    # Take into account the date of the loan, so that future transactions are never considered
    return trans_dev.loc[trans_dev['account_id'] == account_id].loc[trans_dev['date'] < date]

rows_trans_dev = [filter_relevants(trans, row['account_id'], row['date_loan']) for _, row in df.iterrows()]

def count_condition(df, condition):
    return df.loc[condition].shape[0]

def find_credit(subtrans):
    return subtrans.loc[(subtrans['type'] == 'credit') | (subtrans['operation'] == 'credit in cash')]

def find_withdrawal(subtrans):
    # Takes into account the redudancy of the type "withdrawal in cash" which should be an operation, since the type would just be "withdrawal"
    return subtrans.loc[(subtrans['type'] == 'withdrawal') | (subtrans['type'] == 'withdrawal in cash') | (subtrans['operation'] == 'withdrawal in cash') | (subtrans['operation'] == 'credit card withdrawal')]

def signed_mean(subtrans):
    return (
        pd.concat([
            find_credit(subtrans)['amount'],
            -find_withdrawal(subtrans)['amount']
        ])
    ).mean()

# Count categorical values
df['count_trans_credits'] = [find_credit(subtrans).shape[0] for subtrans in rows_trans_dev]
df['count_trans_withdrawals'] = [find_withdrawal(subtrans).shape[0] for subtrans in rows_trans_dev]
df['count_trans_credit_cash'] = [count_condition(subtrans, (subtrans['operation'] == 'credit in cash')) for subtrans in rows_trans_dev]
df['count_trans_withdrawal_cash'] = [count_condition(subtrans, (subtrans['operation'] == 'withdrawal in cash') | (subtrans['type'] == 'withdrawal in cash')) for subtrans in rows_trans_dev]
df['count_trans_withdrawal_card'] = [count_condition(subtrans, (subtrans['operation'] == 'credit card withdrawal')) for subtrans in rows_trans_dev]
df['count_trans_collection_other_bank'] = [count_condition(subtrans, (subtrans['operation'] == 'collection from another bank')) for subtrans in rows_trans_dev]
df['count_trans_remittance_other_bank'] = [count_condition(subtrans, (subtrans['operation'] == 'remittance to another bank')) for subtrans in rows_trans_dev]
df['count_trans_ksymbol_interest_credited'] = [count_condition(subtrans, (subtrans['k_symbol'] == 'interest credited')) for subtrans in rows_trans_dev]
df['count_trans_ksymbol_household'] = [count_condition(subtrans, (subtrans['k_symbol'] == 'household')) for subtrans in rows_trans_dev]
df['count_trans_ksymbol_payment_for_statement'] = [count_condition(subtrans, (subtrans['k_symbol'] == 'payment for statement')) for subtrans in rows_trans_dev]
df['count_trans_ksymbol_insurance_payment'] = [count_condition(subtrans, (subtrans['k_symbol'] == 'insurance payment')) for subtrans in rows_trans_dev]
df['count_trans_ksymbol_sanction_interest_if_negative_balance'] = [count_condition(subtrans, (subtrans['k_symbol'] == 'sanction interest if negative balance')) for subtrans in rows_trans_dev]
df['count_trans_ksymbol_oldage_pension'] = [count_condition(subtrans, (subtrans['k_symbol'] == 'old-age pension')) for subtrans in rows_trans_dev]

# The balance of the account after the last transaction
df['last_trans_balance'] = [subtrans.loc[subtrans['date'] == subtrans['date'].max()]['balance'].values[0] for subtrans in rows_trans_dev]

# Means
df['mean_trans_balance'] = [subtrans['balance'].mean() for subtrans in rows_trans_dev]
df['mean_trans_amount_credit'] = [find_credit(subtrans)['amount'].mean() for subtrans in rows_trans_dev]
df['mean_trans_amount_withdrawal'] = [find_withdrawal(subtrans)['amount'].mean() for subtrans in rows_trans_dev]
# mean of |amount| (both credit and withdrawal count as positive values)
df['mean_trans_amount_absolute'] = [subtrans['amount'].mean() for subtrans in rows_trans_dev]
# signed mean of amount (credit is positive, withdrawal is negative)
df['mean_trans_amount_signed'] = [signed_mean(subtrans) for subtrans in rows_trans_dev]


In [ ]:
for column in ['last_trans_balance', 'mean_trans_balance', 'mean_trans_amount_credit', 'mean_trans_amount_withdrawal', 'mean_trans_amount_absolute', 'mean_trans_amount_signed']:
    print(f"Column {column} has NaNs:", df.isna().any()[column])

In [ ]:
# Since there were no transactions, on average the amount withdrawn should be 0
df['mean_trans_amount_withdrawal'].fillna(0, inplace=True)

In [ ]:
# Drop ids
df.drop(columns=['account_id', 'disp_id_owner', 'client_id_owner', 'type_disp_owner', 'card_id_owner', 'disp_id_disponent', 'client_id_disponent', 'type_disp_disponent', 'card_id_disponent'], inplace=True)

### Extract birthdate and sex from birth_number

In [ ]:
df['owner_male'] = df['birth_number_owner'].apply(lambda x: 0 if int(str(x)[2:4]) > 12 else 1)
df['owner_birthdate'] = df['birth_number_owner'].apply(lambda x: x-5000 if int(str(x)[2:4]) > 12 else x)
df['disponent_male'] = df['birth_number_disponent'].apply(lambda x: (0 if int(str(x)[2:4]) > 12 else 1) if not pd.isna(x) else x)
df['disponent_birthdate'] = df['birth_number_disponent'].apply(lambda x: (x-5000 if int(str(x)[2:4]) > 12 else x) if not pd.isna(x) else x)
df.drop(columns=['amount', 'birth_number_owner', 'birth_number_disponent', 'code ', 'code _owner', 'code _disponent'], inplace=True)


### Rename columns so that they are easier to use and more uniform

In [ ]:
df.rename(columns={
    'loan_id': 'Id', 
    'status': 'Predicted',
    'date_loan': 'loan_date', 
    'duration': 'loan_duration', 
    'payments': 'loan_payments', 
    'district_id': 'account_district_code',
    'name ': 'account_district_name', # removed below
    'region': 'account_district_region',
    'no. of inhabitants': 'account_district_no_inhabitants',
    'no. of municipalities with inhabitants < 499 ': 'account_district_no_municipalities_0_499',
    'no. of municipalities with inhabitants 500-1999': 'account_district_no_municipalities_500_1999',
    'no. of municipalities with inhabitants 2000-9999 ': 'account_district_no_municipalities_2000_9999',
    'no. of municipalities with inhabitants >10000 ': 'account_district_no_municipalities_10000_plus',
    'no. of cities ': 'account_district_no_cities',
    'ratio of urban inhabitants ': 'account_district_ratio_urban_inhabitants',
    'average salary ': 'account_district_average_salary',
    'unemploymant rate \'95 ': 'account_district_unemployment_rate_95',
    'unemploymant rate \'96 ': 'account_district_unemployment_rate_96',
    'no. of enterpreneurs per 1000 inhabitants ': 'account_district_no_enterpreneurs_per_1000_inhabitants',
    'no. of commited crimes \'95 ': 'account_district_no_crimes_95',
    'no. of commited crimes \'96 ': 'account_district_no_crimes_96',
    'frequency': 'account_frequency', 
    'date_account': 'account_date', 
    'owner_male': 'owner_male',
    'owner_birthdate': 'owner_birthdate',
    'district_id_owner': 'owner_district_code',
    'name _owner': 'owner_district_name', # removed below
    'region_owner': 'owner_district_region',
    'no. of inhabitants_owner': 'owner_district_no_inhabitants',
    'no. of municipalities with inhabitants < 499 _owner': 'owner_district_no_municipalities_0_499', 
    'no. of municipalities with inhabitants 500-1999_owner': 'owner_district_no_municipalities_500_1999',
    'no. of municipalities with inhabitants 2000-9999 _owner': 'owner_district_no_municipalities_2000_9999',
    'no. of municipalities with inhabitants >10000 _owner': 'owner_district_no_municipalities_10000_plus',
    'no. of cities _owner': 'owner_district_no_cities',
    'ratio of urban inhabitants _owner': 'owner_district_ratio_urban_inhabitants',
    'average salary _owner': 'owner_district_average_salary',
    'unemploymant rate \'95 _owner': 'owner_district_unemployment_rate_95',
    'unemploymant rate \'96 _owner': 'owner_district_unemployment_rate_96',
    'no. of enterpreneurs per 1000 inhabitants _owner': 'owner_district_no_enterpreneurs_per_1000_inhabitants',
    'no. of commited crimes \'95 _owner': 'owner_district_no_crimes_95',
    'no. of commited crimes \'96 _owner': 'owner_district_no_crimes_96',
    'type_card_owner': 'owner_card_type', 
    'issued_owner': 'owner_card_issued',
    'disponent_male': 'disponent_male',
    'disponent_birthdate': 'disponent_birthdate',
    'district_id_disponent': 'disponent_district_code',
    'name _disponent': 'disponent_district_name', # removed below
    'region_disponent': 'disponent_district_region',
    'no. of inhabitants_disponent': 'disponent_district_no_inhabitants',
    'no. of municipalities with inhabitants < 499 _disponent': 'disponent_district_no_municipalities_0_499',
    'no. of municipalities with inhabitants 500-1999_disponent': 'disponent_district_no_municipalities_500_1999',
    'no. of municipalities with inhabitants 2000-9999 _disponent': 'disponent_district_no_municipalities_2000_9999',
    'no. of municipalities with inhabitants >10000 _disponent': 'disponent_district_no_municipalities_10000_plus',
    'no. of cities _disponent': 'disponent_district_no_cities',
    'ratio of urban inhabitants _disponent': 'disponent_district_ratio_urban_inhabitants',
    'average salary _disponent': 'disponent_district_average_salary',
    'unemploymant rate \'95 _disponent': 'disponent_district_unemployment_rate_95',
    'unemploymant rate \'96 _disponent': 'disponent_district_unemployment_rate_96',
    'no. of enterpreneurs per 1000 inhabitants _disponent': 'disponent_district_no_enterpreneurs_per_1000_inhabitants',
    'no. of commited crimes \'95 _disponent': 'disponent_district_no_crimes_95',
    'no. of commited crimes \'96 _disponent': 'disponent_district_no_crimes_96',
    'type_card_disponent': 'disponent_card_type', # removed below
    'issued_disponent': 'disponent_card_issued',  # removed below
    'count_trans_credits': 'count_trans_credits',
    'count_trans_withdrawals': 'count_trans_withdrawals', 
    'count_trans_credit_cash': 'count_trans_credit_cash',
    'count_trans_withdrawal_cash': 'count_trans_withdrawal_cash', 
    'count_trans_withdrawal_card': 'count_trans_withdrawal_card',
    'count_trans_collection_other_bank': 'count_trans_collection_other_bank',
    'count_trans_remittance_other_bank': 'count_trans_remittance_other_bank',
    'count_trans_ksymbol_interest_credited': 'count_trans_ksymbol_interest_credited',
    'count_trans_ksymbol_household': 'count_trans_ksymbol_household',
    'count_trans_ksymbol_payment_for_statement': 'count_trans_ksymbol_payment_for_statement',
    'count_trans_ksymbol_insurance_payment': 'count_trans_ksymbol_insurance_payment',
    'count_trans_ksymbol_sanction_interest_if_negative_balance': 'count_trans_ksymbol_sanction_interest_if_negative_balance',
    'count_trans_ksymbol_oldage_pension': 'count_trans_ksymbol_oldage_pension', 
    'last_trans_balance': 'last_trans_balance',
    'mean_trans_balance': 'mean_trans_balance', 
    'mean_trans_amount_absolute': 'mean_trans_amount_absolute',
    'mean_trans_amount_credit': 'mean_trans_amount_credit', 
    'mean_trans_amount_withdrawal': 'mean_trans_amount_withdrawal',
    'mean_trans_amount_signed': 'mean_trans_amount_signed'
}, inplace=True)

In [ ]:
# Convert 1 to 0 (positive label) and -1 to 1 (negative label) in target column
df['Predicted'] = df['Predicted'].apply(lambda x: 0 if x == 1 else (1 if x == -1 else np.nan))

### Check that no column uses data from the future

In [ ]:
def check_after_sanity(df, col1, col2):
    cond = (df[col1] > df[col2]) | (df[col1].isna()) | (df[col2].isna())
    notcond = ~cond
    print(f"{col1} > {col2}: {cond.all()} ({notcond.sum()} rows)")

check_after_sanity(df, 'loan_date', 'account_date')
check_after_sanity(df, 'loan_date', 'owner_card_issued')
check_after_sanity(df, 'owner_card_issued', 'account_date')


### Create features based on dates

In [ ]:
def parse_date_single(x):
    if np.isnan(x):
        return x
    return '19' + str(x)[0:2] + '-' + str(x)[2:4] + '-' + str(x)[4:6]

def parse_date(series):
    return pd.to_datetime(series.apply(lambda x: parse_date_single(x)))

def calculate_age(born, now):
    born = parse_date(born)
    now = parse_date(now)
    born_md = born.apply(lambda x: (x.month, x.day))
    now_md = now.apply(lambda x: (x.month, x.day))
    return now.dt.year - born.dt.year - (now_md < born_md)

def calculate_months(born, now):
    born = parse_date(born)
    now = parse_date(now)
    return (now.dt.year - born.dt.year)*12 + (now.dt.month - born.dt.month) - (now.dt.day < born.dt.day)

df['owner_age'] = calculate_age(df['owner_birthdate'], df['loan_date'])
df['account_age_months'] = calculate_months(df['account_date'], df['loan_date'])
df['disponent_age'] = calculate_age(df['disponent_birthdate'], df['loan_date'])

df[df['owner_age'] < 17][['owner_age', 'disponent_age', 'Predicted']]

In [ ]:
df[['owner_age', 'disponent_age', 'account_age_months']].describe()
# There are minors in the dataset


In [ ]:
df[df['owner_age'] < 17][['owner_age', 'disponent_age', 'Predicted']]
# Most minors don't have a disponent or have a disponent who is also a minor

In [ ]:
# These date columns are not needed anymore, as they were replaced by age columns
df.drop(columns=['owner_birthdate', 'disponent_birthdate', 'account_date'], inplace=True)

In [ ]:
corr_plot(df[['owner_male', 'owner_age', 'disponent_male', 'disponent_age',]], size=(3, 3))
# Disponent male and owner male correlated negatively (shared accounts have a man and a woman)
# Ages are correlated positively (account sharers have approximatedly the same age)

### Missing data after processing

In [ ]:
# Missing values after previous processing
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing.to_csv('output/percent_missing.csv')
for row in percent_missing.iteritems():
    print(f"{row[0]}: {row[1]:.4}%")

# Missing values come from disponent information (since many accounts don't have another disponent)
# and from the card issued date (since many clients don't have a card)

In [ ]:
df.drop(columns=['owner_card_issued'], inplace=True)

In [ ]:
disp[disp['disp_id'].isin(cards['disp_id'])]['type'].value_counts()
# All cards are from OWNERS - Remove disponent card information


In [ ]:
# Names are strings (and are equivalent to codes) so they are only useful for human readability, not for the model
df.drop(columns=['disponent_card_type', 'disponent_card_issued', 'account_district_name', 'owner_district_name', 'disponent_district_name'], inplace=True)

In [ ]:
# These work as primary keys, so they will not be able to help the model especially with the little training data we have
df.drop(columns=['account_district_code', 'owner_district_code', 'disponent_district_code'], inplace=True)

In [ ]:
# Replace disponent information by "has_disponent" feature

df['has_disponent'] = df['disponent_male'].apply(lambda x: 0 if np.isnan(x) else 1)
df.drop(columns=[
    'disponent_district_region',
    'disponent_district_no_inhabitants',
    'disponent_district_no_municipalities_0_499',
    'disponent_district_no_municipalities_500_1999',
    'disponent_district_no_municipalities_2000_9999',
    'disponent_district_no_municipalities_10000_plus',
    'disponent_district_no_cities',
    'disponent_district_ratio_urban_inhabitants',
    'disponent_district_average_salary',
    'disponent_district_unemployment_rate_95',
    'disponent_district_unemployment_rate_96',
    'disponent_district_no_enterpreneurs_per_1000_inhabitants',
    'disponent_district_no_crimes_95',
    'disponent_district_no_crimes_96',
    'disponent_male',
    'disponent_age',
], inplace=True)

In [ ]:
df

### Plots of processed data

In [ ]:
sns.countplot(df, x='Predicted', palette='Set1')

# Unbalanced data, sampling probably needed

In [ ]:
corr_plot(df[[
    'account_district_region',
    'account_district_no_inhabitants',
    'account_district_no_municipalities_0_499',
    'account_district_no_municipalities_500_1999',
    'account_district_no_municipalities_2000_9999',
    'account_district_no_municipalities_10000_plus',
    'account_district_no_cities',
    'account_district_ratio_urban_inhabitants',
    'account_district_average_salary',
    'account_district_unemployment_rate_95',
    'account_district_unemployment_rate_96',
    'account_district_no_enterpreneurs_per_1000_inhabitants',
    'account_district_no_crimes_95',
    'account_district_no_crimes_96',
    'owner_district_region',
    'owner_district_no_inhabitants',
    'owner_district_no_municipalities_0_499',
    'owner_district_no_municipalities_500_1999',
    'owner_district_no_municipalities_2000_9999',
    'owner_district_no_municipalities_10000_plus',
    'owner_district_no_cities',
    'owner_district_ratio_urban_inhabitants',
    'owner_district_average_salary',
    'owner_district_unemployment_rate_95',
    'owner_district_unemployment_rate_96',
    'owner_district_no_enterpreneurs_per_1000_inhabitants',
    'owner_district_no_crimes_95',
    'owner_district_no_crimes_96',
]], size=(16, 12))

# Onwer and account district data are highly correlated, so we only need to use one of them


In [ ]:
sns.scatterplot(df, y='mean_trans_balance', x='last_trans_balance', hue='Predicted', palette='Set1')

In [ ]:
sns.scatterplot(df, x='count_trans_credits', y='count_trans_withdrawals', hue='Predicted', palette='Set1')

# credits and withdrawals are correlated

In [ ]:

# 'Number of transactions of sanction interest because of negative balance'
tempdf = df.rename(columns={'count_trans_ksymbol_sanction_interest_if_negative_balance': 'No. transactions sanction interest', 'owner_card_type': 'owner card type', 'Predicted': 'loan status'})
tempdf.dropna(inplace=True)
tempdf['loan status'] = tempdf['loan status'].apply(lambda x: "Didn't pay loan\n(Positive class)" if x == 1 else "Paid loan\n(Negative class)")

plt.figure(figsize=(5, 2.75))
plt.ylim(0, 320)

sns.countplot(tempdf, x='No. transactions sanction interest', hue='loan status', palette='Set1', hue_order=[ "Paid loan\n(Negative class)", "Didn't pay loan\n(Positive class)",])


# Most loaners with sanction interest for negative balance have not paid the loan
# The only exception are the very few loaners with 6 of such transactions, which is an odd outlier

In [ ]:
# Monthly loan payments as a function of balance after last transaction

plt.figure(figsize=(7, 5))

# plt.xlim(0, 9000)
# plt.ylim(0, 500000)

tempdf = df.rename(columns={'last_trans_balance': 'last transaction balance', 'loan_payments': 'loan payments', 'Predicted': 'loan status'})
tempdf.dropna(inplace=True)
tempdf['loan status'] = tempdf['loan status'].apply(lambda x: "Didn't pay loan\n(Positive class)" if x == 1 else "Paid loan\n(Negative class)")
sns.scatterplot(tempdf, x='last transaction balance', y='loan payments', hue='loan status', palette='Set1', hue_order=[ "Paid loan\n(Negative class)", "Didn't pay loan\n(Positive class)",])

# If the loan is big and the balance of the account after its last transaction is low, it’s likely the client will fail to pay the rest.

In [ ]:
sns.scatterplot(df, x='mean_trans_balance', y='loan_payments', hue='Predicted', palette='Set1')

# The previous tendency is not as clear with the mean transaction balance

In [ ]:
# Card types of owners of accounts who made a loan

tempdf = df.rename(columns={'owner_card_type': 'owner card type', 'Predicted': 'loan status'})
tempdf.dropna(inplace=True)
tempdf['loan status'] = tempdf['loan status'].apply(lambda x: "Didn't pay loan\n(Positive class)" if x == 1 else "Paid loan\n(Negative class)")
plt.figure(figsize=(4, 3.5))
plt.ylim(0, 300)

ax = sns.countplot(tempdf, x='owner card type', palette='Set1', order = tempdf['owner card type'].value_counts().index, hue="loan status", hue_order=[ "Paid loan\n(Negative class)", "Didn't pay loan\n(Positive class)",])
for p, label in zip(ax.patches, tempdf['owner card type'].value_counts() *100 / tempdf['owner card type'].count() + 0.011):
    ax.annotate(f"{label:.1f}%", (p.get_x()+0.16, p.get_height()+ 5))

In [ ]:
# Imbalanced distribution of loan status
tempdf = df.rename(columns={'Predicted': 'loan status'})
tempdf['loan status'] = tempdf['loan status'].apply(lambda x: "Didn't pay loan\n(Positive class)" if x == 1 else "Paid loan\n(Negative class)")
plt.figure(figsize=(3, 3.5))
ax = sns.countplot(tempdf, x='loan status', palette='Set1', order = tempdf['loan status'].value_counts().index, hue_order=["Didn't pay loan\n(Positive class)", "Paid loan\n(Negative class)"])
for p, label in zip(ax.patches, tempdf['loan status'].value_counts() *100 / tempdf['loan status'].count() + 0.011):
    ax.annotate(f"{label:.1f}%", (p.get_x()+0.25, p.get_height()+3))
    

In [ ]:
# Loan amount is the product of loan payments and loan duration
tempdf = loans.rename(columns={'payments': 'loan payments', 'duration': 'loan duration', 'amount': 'loan amount'})

plt.figure(figsize=(5, 3))
plt.xlim(0, 9000)
plt.ylim(0, 500000)

sns.scatterplot(tempdf, x='loan payments', y='loan amount', hue='loan duration', palette='Set1')

In [ ]:
# Programatically confirm the above
(loans['amount'] == loans['duration']*loans['payments']).all()

In [ ]:
# Age of owners of accounts who made a loan
tempdf = df.rename(columns={'owner_age': 'owner age'})
plt.figure(figsize=(4, 3))
plt.xlim(10, 65)

ax = sns.histplot(tempdf, x='owner age', kde=True, bins=[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65], palette='Set1',  kde_kws=dict(cut=3))

ax.set_xticks([10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65])
ax.set_xticklabels([10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65])


In [ ]:
tempdistricts = districts.rename({
    'no. of inhabitants': 'no. inhabitants',
    'no. of municipalities with inhabitants < 499 ': 'no. mcp < 499 inab.',
    'no. of municipalities with inhabitants 500-1999': 'no. mcp 500-1999 inab.',
    'no. of municipalities with inhabitants 2000-9999 ': 'no. mcp 2000-9999 inab.',
    'no. of municipalities with inhabitants >10000 ': 'no. mcp >10000 inab.',
    'no. of cities ': 'no. cities',
    'ratio of urban inhabitants ': 'ratio urban inhabitants',
    'average salary ': 'average salary',
    'unemploymant rate \'95 ': 'unemployment rate \'95',
    'unemploymant rate \'96 ': 'unemployment rate \'96',
    'no. of enterpreneurs per 1000 inhabitants ': 'no. enterpreneurs',
    'no. of commited crimes \'95 ': 'no. crimes \'95',
    'no. of commited crimes \'96 ': 'no. crimes \'96',
}, axis=1)

corr_plot(tempdistricts.drop(columns=['code ', 'name ', 'region' ], axis=1))


### Encode categorical features

In [ ]:
# encode string to int
DISTRICT_REGION_ENCODING = {
    'south Moravia': 0,
    'north Moravia': 1,
    'central Bohemia': 2,
    'east Bohemia': 3,
    'Prague': 4,
    'north Bohemia': 5,
    'south Bohemia': 6,
    'west Bohemia': 7
}

CARD_TYPE_ENCODING = {'no card': 0, 'junior': 1, 'classic': 2, 'gold': 3}

ACCOUNT_FREQUENCY_ENCODING = {'issuance after transaction': 0, 'weekly issuance': 1, 'monthly issuance': 2}

df['account_frequency'].replace(ACCOUNT_FREQUENCY_ENCODING, inplace=True)
df['account_district_region'].replace(DISTRICT_REGION_ENCODING, inplace=True)
df['owner_district_region'].replace(DISTRICT_REGION_ENCODING, inplace=True)
df['owner_card_type'].replace(CARD_TYPE_ENCODING, inplace=True)

In [ ]:
# Print all column names
for i in df.columns: 
    print(f"'{i}',")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
df.head()

In [ ]:
df.describe()

In [ ]:
corr_plot(df.drop(columns=['Predicted', 'Id', 'loan_duration']), annot=False)
# Disponent male and owner male correlated negatively (man and woman)

# Save data

In [ ]:
df = df.sort_values(by='loan_date')
competition = df[df['Predicted'].isna()]
data = df[~df['Predicted'].isna()]

if DO_ANALYSIS == "ALL-DATA":
    df.to_csv('output/loan.csv', index=False)

if DO_ANALYSIS == "ALL-DATA" or DO_ANALYSIS == "DEV-DATA":
    data.to_csv('output/loan_dev.csv', index=False)

if DO_ANALYSIS == "ALL-DATA" or DO_ANALYSIS == "COMP-DATA":
    competition.to_csv('output/loan_comp.csv', index=False)
